In [1]:
import pandas as pd
import psycopg2

def extract_credit():
    """Extract data from creditcard table in Postgres"""
    try:
        connection = psycopg2.connect(
            host="localhost",
            dbname="Fraud_Alert",
            user="postgres",
            password="BBK#0027"
        )
        query = "SELECT transaction_id, amount_anomaly, time FROM creditcard;"
        df = pd.read_sql(query, connection)
        connection.close()
        return df
    except Exception as e:
        print("Error during creditcard extract:", e)
        return pd.DataFrame()

def transform_to_alerts(credit_df):
    """Filter anomalies and build fraud_alerts table"""
    # Keep only anomalies
    fraud_df = credit_df[credit_df["amount_anomaly"] == 1].copy()

    # Build fraud_alerts table
    fraud_alerts = pd.DataFrame({
        "id": fraud_df.index + 1,   # can also use fraud_df["transaction_id"]
        "suspicious": "High transaction amount",
        "transaction_id": fraud_df["transaction_id"],
        "alert_time": pd.to_datetime(fraud_df["time"], errors="coerce")
    })
    return fraud_alerts

def load_alerts(fraud_alerts):
    """Load alerts into fraud_alerts table in Postgres"""
    try:
        connection = psycopg2.connect(
            host="localhost",
            dbname="Fraud_Alert",
            user="postgres",
            password="BBK#0027"
        )
        con = connection.cursor()

        for _, row in fraud_alerts.iterrows():
            con.execute("""
                INSERT INTO fraud_alerts(id, suspicious, transaction_id, alert_time)
                VALUES (%s, %s, %s, %s)
            """, (int(row["id"]), row["suspicious"], int(row["transaction_id"]), row["alert_time"]))
        
        connection.commit()
        con.close()
        connection.close()
        print("Data loaded successfully into fraud_alerts table.")

    except Exception as e:
        print("Error during fraud_alerts load:", e)

if __name__ == "__main__":
    # Extract
    credit_df = extract_credit()

    # Transform
    fraud_alerts = transform_to_alerts(credit_df)

    # Load
    load_alerts(fraud_alerts)

C:\Users\daved\AppData\Local\Temp\ipykernel_17928\3523327928.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


Data loaded successfully into fraud_alerts table.
